In [1]:
!pip install lxml
!pip3 install pandas

     |████████████████████████████████| 5.5MB 10.8MB/s eta 0:00:01     |██████████                      | 1.7MB 10.8MB/s eta 0:00:01
     |████████████████████████████████| 9.7 MB 10.9 MB/s eta 0:00:01
     |████████████████████████████████| 15.4 MB 20.4 MB/s eta 0:00:01     |███████████████████████████     | 13.0 MB 20.4 MB/s eta 0:00:01
     |████████████████████████████████| 510 kB 59.1 MB/s eta 0:00:01


In [2]:
import pandas as pd
import lxml
import numpy as np

In [3]:
toronto_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' # The Wikipedia page containing Toronto data

In [4]:
dfs = pd.read_html(toronto_url) # Scraping the tables from the page into series

In [5]:
df = dfs[0] # Selecting the correct series

In [6]:
df_stacked = pd.Series(df.values.ravel('F')) # Converting into rows for each postal code

In [7]:
df_stacked.shape # Checking the shape, if 180 it is correct

(180,)

In [8]:
df1 = df_stacked.to_frame() # Convert to dataframe

In [9]:
df1['PostalCode'] = df1[0].str[:3] # Extract postal code as column

In [10]:
df1['rest'] = df1[0].str[3:] #Extract rest of data as column

In [11]:
df1[['Borough','Neighborhood']] = df1.rest.str.split('\(|\)', expand=True).iloc[:,[0,1]] # Split into borough and neighborhood colums

In [12]:
df1.drop(0, axis=1, inplace=True)
df1.drop('rest', axis=1, inplace=True) # Dropping redundant columns

In [13]:
df1['Neighborhood'] = df1['Neighborhood'].str.replace(' /',',') # Replacing / with commas for separating neighborhoods

In [14]:
df1 = df1[df1["Borough"].str.contains("Not assigned")==False] # Drop unassigned boroughs

In [15]:
df1['Neighborhood'] = np.where(df1['Neighborhood'] == 'None', df1['Borough'], df1['Neighborhood']) # Change 'None' neighbourhoods to borough name
df1

,PostalCode,Borough,Neighborhood
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
...,...,...,...
170,M9N,York,Weston
171,M9P,Etobicoke,Westmount
172,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
175,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [16]:
df1.shape

(103, 3)

In [17]:
df_geo = pd.read_csv("https://cocl.us/Geospatial_data") # Import geographical coordinates of Toronto postal codes

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [19]:
df_geo.rename(columns={"Postal Code": "PostalCode"}, inplace=True) # Change column name for merging

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [20]:
df2 = pd.merge(df1, df_geo) # Add coordinates to original dataframe
df2

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
